In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver #IMPORTAMOS UN MODULO PARA ALMACENAR MEMORIA
from langgraph.graph import START, MessagesState, StateGraph #IMPORTAMOS MODULOS PARA NUESTRO GRAFO
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder #IMPORTAMOS MODULOS PARA EL PROMPT
load_dotenv()
model = ChatOpenAI(model="gpt-4o-mini")

#SE CREA UN GRAFO PARA MANEJAR EL FLUJO DE LA CONVERSACION
workflow = StateGraph(state_schema=MessagesState)

#DEFINIMOS EL PROMPT "MessagesPlaceholder(variable_name="messages")" ES DINAMICO, SE VA INSERTANDO EL HISTORIAL DE LA CONVERSACION
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres una IA especializada en Pokémon, desde la primera hasta la novena generación. "
            "Sabes el tipo del Pokémon, sus stats, sus debilidades, sus habilidades ocultas,, "
            "los ataques que aprende por nivel y por MT según la generación que se te pida. Se preciso en la respuesta y responde solamente lo que se te pide en un maximo de 2 lineas",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

#FUNCION QUE RECIBE LOS MENSAJES QUE SE HAN ACOMULADO, A PARTIR DE LA CONVERSACION ENTERA RETORNA UN NUEVO MENSAJE
def call_model(state: MessagesState):
    formatted = prompt_template.invoke({"messages": state["messages"]}) #INYECTAMOS EL HISTORIAL EN EL PROMPT
    response = model.invoke(formatted)  # usar el prompt formateado
    return {"messages": [response]}     # devolver como lista para que LangGraph lo agregue al estado

#AÑADE NODOS CON EL NOMBRE MODEL EL CUAL EJECUTA CALL_MODEL
workflow.add_node("model", call_model)

#CONECTA LOS NODOS A PARTIR DEL PRIMERO HASTA EL ULTIMO CREADO ANTERIORMENTE
workflow.add_edge(START, "model")

#SE CREA UN ALMACENADOR DE MEMORIA
memory = MemorySaver()

#SE CREA UNA APP QUE COMPILA TODO EL GRAFO AGREGANDOLE MEMORIA
app = workflow.compile(checkpointer=memory)

#ASIGNAMOS UNA CONFIGURACION CON UN THREAD (THREAD ES EL ID QUE NOS SIRVE PARA OBTENER LOS MENSAJES GUARDADOS)
config = {"configurable": {"thread_id": "abc123"}}

#PARA CORRER EN CONSOLA
if __name__ == "__main__":
    print("¡Bienvenido al chat Pokémon! (Escribe 'salir' para terminar)\n")
    while True:
        user_input = input("Tú: ")
        if user_input.lower() in ["salir", "exit", "quit"]:
            print("👋 ¡Hasta luego!")
            break
        if not user_input.strip():
            continue
        print(f"Tú: {user_input}")
        out = app.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
        ai_response = out["messages"][-1].content.strip()
        print(f"AI: {ai_response}\n")




¡Bienvenido al chat Pokémon! (Escribe 'salir' para terminar)

Tú: que tipo es blaziken
AI: Blaziken es de tipo Fuego y Lucha.

Tú: sus stats
AI: Blaziken tiene los siguientes stats base: HP 80, Ataque 120, Defensa 70, Ataque Especial 110, Defensa Especial 70, Velocidad 80.

Tú: que tipo es swampert
AI: Swampert es de tipo Agua y Tierra.

Tú: sus stats
AI: Swampert tiene los siguientes stats base: HP 100, Ataque 110, Defensa 90, Ataque Especial 85, Defensa Especial 90, Velocidad 60.

Tú: por que pokemon te pregunte
AI: Me preguntaste por Swampert.

👋 ¡Hasta luego!


In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver  
from langgraph.graph import START, StateGraph        
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  
#NOS SIRVE PARA MANEJAR EL TIPADO SECUENCIA Y AGREGACION DE MENSAJES
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from uuid import uuid4

load_dotenv()
model = ChatOpenAI(model="gpt-4o-mini")

#DEFINIMOS UNA CLASE QUE NOS PERMITA MANEJAR DE MANERA MAS AUTOMATICA LA MEMORIA
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]  #ACOMULAMOS LOS MENSAJES
    language: str                                             #IDIMOA EN QUE RESPONDERA

workflow = StateGraph(state_schema=State)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres una IA especializada en Pokémon, desde la primera hasta la novena generación. "
            "Responde de forma precisa y directa en {language}. "
            "Si el usuario pide estadísticas o movimientos, entrégalos completos y compactos.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def call_model(state: State):
    prompt = prompt_template.invoke(state)          # INYECTAMOS EL HISTORIAL Y EL 'language' EN EL PROMPT
    response = model.invoke(prompt)                
    return {"messages": [response]}                


workflow.add_node("model", call_model)
workflow.add_edge(START, "model")
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

if __name__ == "__main__":
    print("¡Bienvenido al chat Pokémon! (Escribe 'salir' para terminar)\n")
    while True:
        user_input = input("Tú: ")
        if user_input.lower() in ["salir", "exit", "quit"]:
            print("👋 ¡Hasta luego!")
            break
        if not user_input.strip():
            continue
        print(f"Tú: {user_input}")  
        out = app.invoke(
            {
                "messages": [{"role": "user", "content": user_input}],
                "language": "es",
            },
            config=config,
        )
        ai_response = out["messages"][-1].content.strip()
        print(f"AI: {ai_response}\n")

¡Bienvenido al chat Pokémon! (Escribe 'salir' para terminar)

Tú: que pokemone ste pregunte
AI: Por favor, proporciona el nombre del Pokémon del que deseas información y estaré encantado de ayudarte.

Tú: que tipo es blaziken
AI: Blaziken es un Pokémon de tipo Fuego y Lucha.

Tú: Escape
AI: Escape es un movimiento de tipo Normal. Aquí tienes la información completa:

- **Tipo:** Normal
- **Categoría:** Estado
- **PP:** 20
- **Precisión:** -
- **Efecto:** Permite al usuario escapar de la batalla, incluso si no puede huir normalmente. En batallas contra otros entrenadores, no se puede usar.

Si necesitas más información, ¡pregunta!

👋 ¡Hasta luego!


In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver  
from langgraph.graph import START, StateGraph        
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import SystemMessage, trim_messages #IMPORTAMOS ESTE MODULO PARA QUE NO RECUERDE TODA LA MEMORIA ENTERA
# ESTO PUEDE GENERAR PROBLEMAS YA QUE PUEDE TRAER MUCHA INFORMACION Y EL MODELO DEMORARIA EN CARGAR.

load_dotenv()
model = ChatOpenAI(model="gpt-4o-mini")

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str 

workflow = StateGraph(state_schema=State)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres una IA especializada en Pokémon, desde la primera hasta la novena generación. "
            "Responde de forma precisa y directa en {language}. "
            "Si el usuario pide estadísticas o movimientos, entrégalos completos y compactos.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

trimmer = trim_messages(
    max_tokens=65,        # tope de tokens para TODO el historial
    strategy="last",      # si hay que cortar dentro de un mensaje parcial, toma los ÚLTIMOS tokens
    token_counter=model,  # usa el contador de tokens del modelo para medir
    include_system=True,  # preserva el mensaje de sistema al recortar
    allow_partial=False,  # NO incluye fragmentos de un mensaje; solo mensajes completos
    start_on="human",     # asegura que la secuencia resultante comience con un mensaje humano
)

def call_model(state: State):
    # Muestra cuántos mensajes hay antes de recortar (útil para depurar en consola/Jupyter)
    print(f"Messages before trimming: {len(state['messages'])}")
    # Aplica el recortador configurado (max_tokens, estrategia, etc.) sobre el historial
    trimmed_messages = trimmer.invoke(state["messages"])
    # Muestra cuántos mensajes quedaron tras el recorte
    print(f"Messages after trimming: {len(trimmed_messages)}")
    # Lista (en texto) qué mensajes permanecen y su contenido
    print("Remaining messages:")
    for msg in trimmed_messages:
        # Imprime el tipo de mensaje (HumanMessage, AIMessage, SystemMessage) y su contenido
        print(f"  {type(msg).__name__}: {msg.content}")
    # Construye el prompt final usando SOLO los mensajes recortados + el idioma del estado
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    # Llama al modelo con el prompt formateado
    response = model.invoke(prompt)
    # Devuelve la respuesta como lista para que LangGraph la agregue al estado
    return {"messages": [response]}             

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

if __name__ == "__main__":
    print("¡Bienvenido al chat Pokémon! (Escribe 'salir' para terminar)\n")
    while True:
        user_input = input("Tú: ")
        if user_input.lower() in ["salir", "exit", "quit"]:
            print("👋 ¡Hasta luego!")
            break
        if not user_input.strip():
            continue
        print(f"Tú: {user_input}")  
        out = app.invoke(
            {
                "messages": [{"role": "user", "content": user_input}],
                "language": "es",
            },
            config=config,
            )
        ai_response = out["messages"][-1].content.strip()
        print(f"AI: {ai_response}\n")

¡Bienvenido al chat Pokémon! (Escribe 'salir' para terminar)

Tú: QUE TIPO ES BLAZIKEN
Messages before trimming: 1
Messages after trimming: 1
Remaining messages:
  HumanMessage: QUE TIPO ES BLAZIKEN
AI: Blaziken es de tipo Fuego y Lucha.

Tú: QUE TIPO E SSWAMPERT
Messages before trimming: 3
Messages after trimming: 3
Remaining messages:
  HumanMessage: QUE TIPO ES BLAZIKEN
  AIMessage: Blaziken es de tipo Fuego y Lucha.
  HumanMessage: QUE TIPO E SSWAMPERT
AI: Swampert es de tipo Agua y Tierra.

Tú: QUE TIPO ES PALKIA
Messages before trimming: 5
Messages after trimming: 3
Remaining messages:
  HumanMessage: QUE TIPO E SSWAMPERT
  AIMessage: Swampert es de tipo Agua y Tierra.
  HumanMessage: QUE TIPO ES PALKIA
AI: Palkia es de tipo Agua y Dragón.

Tú: cual fue el primer pokemon q que te pregunte
Messages before trimming: 7
Messages after trimming: 3
Remaining messages:
  HumanMessage: QUE TIPO ES PALKIA
  AIMessage: Palkia es de tipo Agua y Dragón.
  HumanMessage: cual fue el primer pok

In [ ]:
# ESTO NOS PERMITE CONSERVAR TODO EL HISTORIAL DE FORMA RESUMIDA
# from typing import Sequence
# from typing_extensions import Annotated, TypedDict
# from langchain_core.messages import BaseMessage
# from langgraph.graph.message import add_messages
# from langchain_core.prompts import ChatPromptTemplate

# # 1) Extiende tu State
# class State(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], add_messages]
#     language: str
#     summary: str  # <-- nuevo

# # 2) Añade el summary al prompt
# prompt_template = ChatPromptTemplate.from_messages([
#     ("system",
#      "Eres experto en Pokémon. Responde en {language}.\n"
#      "Resumen previo de la conversación:\n{summary}"),
#     MessagesPlaceholder("messages"),
# ])

# # 3) Pequeña utilidad para actualizar el resumen
# def update_summary(old_summary: str, dropped: Sequence[BaseMessage]) -> str:
#     summarize_prompt = ChatPromptTemplate.from_messages([
#         ("system", "Resume en 3–4 líneas los puntos relevantes para continuar la conversación."),
#         ("human", "Resumen previo:\n{old}\n\nFragmentos nuevos:\n{new}"),
#     ])
#     new_text = "\n".join(m.content for m in dropped if getattr(m, "content", None))
#     msg = summarize_prompt.invoke({"old": old_summary, "new": new_text})
#     return model.invoke(msg).content.strip()

# # 4) Integra con tu trimmer dentro de call_model
# def call_model(state: State):
#     print(f"Messages before trimming: {len(state['messages'])}")
#     trimmed = trimmer.invoke(state["messages"])
#     print(f"Messages after trimming: {len(trimmed)}")
#     print("Remaining messages:")
#     for m in trimmed:
#         print(f"  {type(m).__name__}: {m.content}")

#     # Si recortaste, actualiza el resumen con lo que se cayó
#     if len(trimmed) < len(state["messages"]):
#         dropped = state["messages"][:len(state["messages"]) - len(trimmed)]
#         new_summary = update_summary(state.get("summary", ""), dropped)
#     else:
#         new_summary = state.get("summary", "")

#     # Construye el prompt final con mensajes recortados + summary + language
#     prompt = prompt_template.invoke({
#         "messages": trimmed,
#         "language": state["language"],
#         "summary": new_summary,
#     })
#     response = model.invoke(prompt)
#     return {"messages": [response], "summary": new_summary}